In [80]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

In [81]:
caffe.set_mode_cpu()
net = caffe.Net('../models/swimmers_fullconv/swimmers_fullconv.prototxt', 
                '../models/swimmers_fullconv/out_new.binaryproto', caffe.TEST)
net.reshape()

In [83]:
# find appearances weight blob
appearance_weights = net.params['appearances'][0].data
print appearance_weights.shape

(13, 392, 1, 1)


In [84]:
B = np.where(appearance_weights[2]==1)[0]
print len(B), B

54 [  2  15  25  40  50  54  72  86  95 101 106 116 125 140 149 161 177 182
 192 199 203 216 219 225 235 249 252 260 262 268 273 274 282 284 293 295
 300 310 313 323 324 325 327 329 332 339 342 347 358 362 363 366 377 383]


In [69]:
appearance_path = '/data/share/idpr_code/appearanceclasses.txt'

def read_appearances(file): 
    with open(appearance_path) as f:
        content = f.readlines()

    joints = {}

    for line in content:
        line = line.replace('\n', '')
        splitted = line.split("\t")
        joint = int(splitted[0])
        num = int(splitted[1])
        joints[joint] = map(int, splitted[2:2+num]) 

    return joints

In [70]:
joints = read_appearances(appearance_path)

for j in xrange(appearance_weights.shape[0]):
    for index in joints[j]:
        appearance_weights[j, index, 0, 0] = 1

In [72]:
B = np.where(appearance_weights[0]==1)[0]
print len(B), B

9 [  0  13  38  80  84 120 138 175 345]


In [75]:
idprclasses_path = '/data/share/idpr_code/idprclasses.txt'

def read_idprclasses(file): 
    with open(appearance_path) as f:
        content = f.readlines()

    classes = {}
    idx = 0

    for line in content:
        line = line.replace('\n', '')
        splitted = line.split("\t")
        from_ = int(splitted[0])
        to_ = int(splitted[1])
        cluster_ = int(splitted[2])
        n_ = int(splitted[3])
        classes[idx] = map(int, splitted[4:4+n_]) 
        idx+=1

    return classes

In [76]:
classes = read_idprclasses(idprclasses_path)

In [79]:
print len(classes)

152


In [86]:
# find idprclasses weight blob
idprclasses_weights = net.params['idprclasses'][0].data
print idprclasses_weights.shape

(152, 392, 1, 1)


In [87]:
classes = read_idprclasses(idprclasses_path)

for j in xrange(idprclasses_weights.shape[0]):
    for index in classes[j]:
        idprclasses_weights[j, index, 0, 0] = 1


In [89]:
B = np.where(idprclasses_weights[7]==1)[0]
print len(B), B

25 [ 14  49  71 115 130 134 139 174 176 207 213 218 239 244 256 292 306 316
 317 326 328 349 360 361 373]


In [90]:
# save changes to new binaryproto
net.save('../models/swimmers_fullconv/out_new2.binaryproto')